In [117]:
# resetting the entire session
%reset -f  
# Iris data-set
from sklearn.datasets import load_iris

# Load Iris dataset
iris = load_iris()
X, y = iris.data, iris.target

In [118]:
# activate credencial, one time, if needed
#!/dataVault/activate_IBMProvider.py 

In [119]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Normalize data
scaler = MinMaxScaler(feature_range=(-1, 1))
X = scaler.fit_transform(X)

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.shape, y_train.shape

((120, 4), (120,))

### Quantum Machine Learning Model
we’ll train a variational quantum classifier (VQC), available in Qiskit Machine Learning 
 Two of its central elements are the feature map and ansatz.

In [120]:
# features circuit
from qiskit.circuit.library import ZZFeatureMap
num_features = X.shape[1]
feature_circ = ZZFeatureMap(feature_dimension=num_features, reps=1)
feature_circ.decompose().draw(output='text', fold=-1)

┌───┐┌─────────────┐                                                                                                                                                                                                                            
q_0: ┤ H ├┤ P(2.0*x[0]) ├──■────────────────────────────────────■────■────────────────────────────────────■─────────■─────────────────────────────────────────■──────────────────────────────────────────────────────────────────────────────────────
     ├───┤├─────────────┤┌─┴─┐┌──────────────────────────────┐┌─┴─┐  │                                    │         │                                         │                                                                                      
q_1: ┤ H ├┤ P(2.0*x[1]) ├┤ X ├┤ P(2.0*(π - x[0])*(π - x[1])) ├┤ X ├──┼────────────────────────────────────┼────■────┼────────────────────────────────────■────┼────■────────────────────────────────────■────────────────────────────────────────────
     ├───┤├─────────────┤└───┘└──────────────────────────────┘└───┘┌─┴─┐┌──────────────────────────────┐┌─┴─┐┌─┴─┐  │  ┌──────────────────────────────┐┌─┴─┐  │    │                                    │                                            
q_2: ┤ H ├┤ P(2.0*x[2]) ├──────────────────────────────────────────┤ X ├┤ P(2.0*(π - x[0])*(π - x[2])) ├┤ X ├┤ X ├──┼──┤ P(2.0*(π - x[1])*(π - x[2])) ├┤ X ├──┼────┼────────────────────────────────────┼────■────────────────────────────────────■──
     ├───┤├─────────────┤                                          └───┘└──────────────────────────────┘└───┘└───┘┌─┴─┐├──────────────────────────────┤└───┘┌─┴─┐┌─┴─┐┌──────────────────────────────┐┌─┴─┐┌─┴─┐┌──────────────────────────────┐┌─┴─┐
q_3: ┤ H ├┤ P(2.0*x[3]) ├─────────────────────────────────────────────────────────────────────────────────────────┤ X ├┤ P(2.0*(π - x[0])*(π - x[3])) ├─────┤ X ├┤ X ├┤ P(2.0*(π - x[1])*(π - x[3])) ├┤ X ├┤ X ├┤ P(2.0*(π - x[2])*(π - x[3])) ├┤ X ├
     └───┘└─────────────┘                                                                                         └───┘└──────────────────────────────┘     └───┘└───┘└──────────────────────────────┘└───┘└───┘└──────────────────────────────┘└───┘

In [121]:
# Anzatz circuit
# This circuit has 16 parameters named θ[0], ..., θ[15]. These are the trainable weights of the classifier.
from qiskit.circuit.library import EfficientSU2
nReps=1
ansatz_circ=EfficientSU2(num_qubits=num_features, reps=nReps, entanglement='linear')
ansatz_circ.decompose().draw(output='text', fold=-1)

┌──────────┐┌──────────┐     ┌──────────┐┌───────────┐                          
q_0: ┤ Ry(θ[0]) ├┤ Rz(θ[4]) ├──■──┤ Ry(θ[8]) ├┤ Rz(θ[12]) ├──────────────────────────
     ├──────────┤├──────────┤┌─┴─┐└──────────┘└┬──────────┤┌───────────┐             
q_1: ┤ Ry(θ[1]) ├┤ Rz(θ[5]) ├┤ X ├─────■───────┤ Ry(θ[9]) ├┤ Rz(θ[13]) ├─────────────
     ├──────────┤├──────────┤└───┘   ┌─┴─┐     └──────────┘├───────────┤┌───────────┐
q_2: ┤ Ry(θ[2]) ├┤ Rz(θ[6]) ├────────┤ X ├──────────■──────┤ Ry(θ[10]) ├┤ Rz(θ[14]) ├
     ├──────────┤├──────────┤        └───┘        ┌─┴─┐    ├───────────┤├───────────┤
q_3: ┤ Ry(θ[3]) ├┤ Rz(θ[7]) ├─────────────────────┤ X ├────┤ Ry(θ[11]) ├┤ Rz(θ[15]) ├
     └──────────┘└──────────┘                     └───┘    └───────────┘└───────────┘

In [122]:
from matplotlib import pyplot as plt
from IPython.display import clear_output
import time

loss_hist = []
plt.rcParams["figure.figsize"] = (12, 6)

def callback_graph(weights, loss):
    clear_output(wait=True)
    loss_hist.append(loss)
    nIter=len(loss_hist)
    if nIter==1:
        plt.tstart=time.time()
        txt='iter=1'
    else:
        elaT = (time.time() - plt.tstart)/60.
        speed=60*(nIter)/elaT # I missed 0-th iteration
        txt='iteration:%d  elaT=%.1f (min), speed %.1f iter/h,  x-entropy=%.2f'%(nIter,elaT,speed,loss)
    print('done:',txt)

    plt.title("Objective function, "+txt)
    plt.xlabel("Iteration")
    plt.ylabel("Corssentropy")
    plt.plot(range(len(loss_hist)), loss_hist)
    plt.grid(color='b', linestyle='--', linewidth=0.5)
    plt.show()

### Training of Qiskit NeuralNetworkClassifier 

In [123]:
if 1:  # backend = density matrix simulator
    from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Options, Session
    #load the service
    service = QiskitRuntimeService(channel = 'ibm_quantum')
    backend = service.get_backend('ibmq_qasm_simulator') # change this for a real execution
    # create the program for samplig results on a backend
    options = Options()
    options.resilience_level = 0  # no need to post-process for ideal backend
    options.execution.shots =1000
    session = Session(backend=backend)
    sampler = Sampler(session=session, options=options)
    #sampler = Sampler(backend, options) #if you do not want to use sessions
else:  # backend= state vector 
    assert sum(sum(lst) for lst in midMeasLL) ==0  # it can't handle mid-circui measurements
    from qiskit.primitives import Sampler  # state vector
    sampler = Sampler()

# To make the training process faster, we choose a gradient-free optimizer.
from qiskit.algorithms.optimizers import COBYLA
nIter=17
optimizer = COBYLA(maxiter=nIter)

In [124]:
# construct quantum circuit  (only for inspection)
from qiskit import QuantumCircuit
num_cregs = len(ansatz_circ.clbits) # number of classical registers in the ansatz
circuit = QuantumCircuit(num_features,num_cregs)
circuit.append(feature_circ, range(num_features))
circuit.append(ansatz_circ, range(num_features),range(num_cregs))
circuit.decompose().draw(output="text", fold=-1)

┌────────────────────────────────────┐┌──────────────────────────────────────────────────────────────────────────────────────────────────────┐
q_0: ┤0                                   ├┤0                                                                                                     ├
     │                                    ││                                                                                                      │
q_1: ┤1                                   ├┤1                                                                                                     ├
     │  ZZFeatureMap(x[0],x[1],x[2],x[3]) ││  EfficientSU2(θ[0],θ[1],θ[2],θ[3],θ[4],θ[5],θ[6],θ[7],θ[8],θ[9],θ[10],θ[11],θ[12],θ[13],θ[14],θ[15]) │
q_2: ┤2                                   ├┤2                                                                                                     ├
     │                                    ││                                                                                                      │
q_3: ┤3                                   ├┤3                                                                                                     ├
     └────────────────────────────────────┘└──────────────────────────────────────────────────────────────────────────────────────────────────────┘

### SamplerQNN

In [125]:
def recoLabel(mval):  # mval=int(bistrings)
    if 1: x=mval%3 # --> 0,1,2
    if 0:
        mbits = bin(mval)[2:]
        hw=mbits.count('1')
        x=hw%3
    return x
numLabels = 3  # corresponds to the number of classes, must cover all  possible outcomes of recoLabel(.)

In [130]:
# construct QNN
# see https://github.com/qiskit-community/qiskit-machine-learning/blob/main/docs/tutorials/01_neural_networks.ipynb
from qiskit_machine_learning.algorithms.classifiers import NeuralNetworkClassifier
from qiskit_machine_learning.neural_networks import SamplerQNN

# SamplerQNN directly consumes samples from measuring the quantum circuit, it does not require a custom observable.
sampler_qnn = SamplerQNN(
    circuit=circuit.decompose(), #we decompose the circuit because samplerqnn will check if the measurements are present in the circuit
    input_params=feature_circ.parameters,
    weight_params=ansatz_circ.parameters,
    interpret=recoLabel, # interpret  bitstrings.
    output_shape=numLabels, # must match to interpreter
    sampler=sampler
)
'''These output samples are interpreted by default as the probabilities of measuring the integer index c
orresponding to a bitstring. However, the SamplerQNN also allows us to specify an interpret function 
to post-process the samples. This function should be defined so that it takes a measured integer 
(from a bitstring) and maps it to a new value, i.e. non-negative integer.
'''

# construct classifier
model = NeuralNetworkClassifier(
    neural_network=sampler_qnn, optimizer=optimizer, callback=callback_graph, loss='cross_entropy'
)

In [129]:
# create empty array for callback to store evaluations of the objective function
# set figsize
plt.rcParams["figure.figsize"] = (12, 6)
loss_hist = []
start = time.time()
# fit classifier to data
model.fit(X_train, y_train)
elapsed = time.time() - start
print(f"Training time: {round(elapsed)} seconds, last value:%.2f"%loss_hist[-1])

#close your session
#session.close()

capi_return is NULL
Call-back cb_calcfc_in__cobyla__user__routines failed.


KeyboardInterrupt: 

### predict for test samples

In [ ]:
# Predict labels for the test data
y_pred =model.predict(X_test)
# Compute the confusion matrix
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_test, y_pred)
#print(conf_matrix)
print('\nconfusion matrix, test samples:%d'%(y_pred.shape[0]))
for i,rec  in enumerate(conf_matrix):
    print('true:%d  reco:%s'%(i,rec))

In [ ]:
# Now we check out how well our classical model performs. 
# mean accuracy of the classifier
train_score_c4 = model.score(X_train, y_train)
test_score_c4 = model.score(X_test, y_test)

print(f"mean accuracy  on  training dataset: {train_score_c4:.2f}")
print(f"mean accuracy on test dataset:     {test_score_c4:.2f}")

In [ ]:
# run full circuit for 
weights=model.weights
'weihts:',weights.shape, weights

In [ ]:
nSamp=20
samples=X_test[:nSamp]
labels=y_test[:nSamp]
pred=model.predict(samples)
nok=0
for p,t in zip(pred,labels):
    print(p,t,p==t)
    nok+=p==t
print('avr prob=%.2f'%(nok/nSamp))

In [ ]:
#session.close()

In [ ]:
model.save("qnn_classifier.model")

In [ ]:
model2 = NeuralNetworkClassifier.load("qnn_classifier.model")

In [ ]:
options.execution.shots =4000
sampler = Sampler(session=session, options=options)
model2.warm_start = True
model2.neural_network.sampler = sampler
model2.optimizer = COBYLA(maxiter=nIter//2)
model2.callback=callback_graph

In [ ]:
# clear objective value history
loss_hist = []

start = time.time()
# fit classifier to data
model2.fit(X_train, y_train)
elapsed = time.time() - start
print(f"Training time: {round(elapsed)} seconds, last value:%.2f"%loss_hist[-1])

# return to default figsize
plt.rcParams["figure.figsize"] = (6, 4)


In [ ]:
# Predict labels for the test data
y_pred =model2.predict(X_test)
# Compute the confusion matrix
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_test, y_pred)
#print(conf_matrix)
print('\nconfusion matrix, test samples:%d'%(y_pred.shape[0]))
for i,rec  in enumerate(conf_matrix):
    print('true:%d  reco:%s'%(i,rec))